We will work with the data provided by the [Open Data portal of Italy](http://www.datiopen.it) that aggregates datasets published by the Italian government, the Regions and Provinces of Italy and the City Houses. We will work with the dataset produced by the Region of Lazio regarding the bars & restaurants that operate in the region in the period 2007-2013. The data is originally provided by the [Regional Observatory of Lazio](http://www.osservatoriocommercio.lazio.it/). The dataset is available in various different formats, here we will work with the one in JSON format:
* [Regione Lazio Regione Lazio - Pubblici esercizi permanenti (Bar-Ristoranti)](http://www.datiopen.it/it/opendata/Regione_Lazio_Pubblici_esercizi_permanenti_Bar_Ristoranti_)

The Open Data portal is developed on the [CKAN platform](https://ckan.org/), a data management system that makes data accessible – by providing tools to streamline publishing, sharing, finding and using data. Through the *Scarica* tab you can download the dataset in JSON format (under the section *Esporta in altri formati*). For convenience, rename the file to *dataset.json*.

The dataset provides the following information:
- Municipality (ccomune)
- Province (cprovincia)
- Year (canno)
- Bars
 * Total (cbar_totale)
 * New Openings (cbar_aperture)
 * Closed (cbar_chiusure)
 * Subingressi (cbar_subingressi)
- Restaurants
 * Total (cristoranti_totale)
 * New Openings (cristoranti_aperture)
 * Closed (cristoranti_chiusure)
 * Subingressi (cristoranti_subingressi)
- Bar-Restaurants
 * Total (cbar_ristoranti_totale)
 * New Openings (cbar_ristoranti_aperture)
 * Closed (cbar_ristoranti_chiusure)
 * Subingressi (cbar_ristoranti_subingressi)
  
### Load Datasets in JSON format

The files retrieved follows a JSON format. 

We will use the [JSON encoder and decoder](https://docs.python.org/3/library/json.html) a standard python package as defined in [W3C School on JSON](https://www.w3schools.com/js/js_json_intro.asp). 

We load all the contents of the file into a string variable *data*.

In [1]:
fo = open('dataset.json', 'r')
data = fo.read()
fo.close()

Essentially all the contents of the file are loaded into the available memory of our machine.

In [2]:
len(data)

1216003

We use the [loads](https://docs.python.org/3/library/json.html#encoders-and-decoders) standard decoder that converts the JSON string sequence into a standard list object where each row of the list contains a dictionary object.

In [3]:
import json
dataset = json.loads(data)

In [4]:
type(dataset)

list

The dataset contains a total of 2646 rows.

In [5]:
len(dataset)

2646

In this format we can access each row directly:

In [6]:
dataset[0]

{'canno': '2007',
 'caperture': '0',
 'cbar_aperture': '',
 'cbar_chiusure': '',
 'cbar_ristoranti_aperture': '',
 'cbar_ristoranti_chiusure': '',
 'cbar_ristoranti_subingressi': '',
 'cbar_ristoranti_totale': '',
 'cbar_subingressi': '',
 'cbar_totale': '',
 'cchiusure': '0',
 'ccomune': 'Acquapendente',
 'cprovincia': 'VITERBO',
 'cristoranti_aperture': '',
 'cristoranti_chiusure': '',
 'cristoranti_subingressi': '',
 'cristoranti_totale': '',
 'csubingressi': '1',
 'ctotale': '32'}

## Storing data in a Document Database

We now wish to store the contents of the dataset into a document database where each row becomes a separate document. We will use the [MongoDB](https://www.mongodb.com/what-is-mongodb) and the Database-as-a-service provider [mLab](https://mlab.com/).

Using the web interface provided by mLab:
1. Create a new account under mLab
2. Create a new Database utilizing the 500MB sandbox provided by MLab without any charges.

### Activate Data API Access

We will access the mongoDB database provided by mLab through the [API (Application Programmers Interface)](http://docs.mlab.com/data-api/). To do so we need to enable the API access and acquire a API Key. Once again, through the web interface provided by mLab:

1. Follow the link by clicking on your username, on the top-right of the main web page.
2. Copy the API key provided by mLab account - this is unique for your account, 
3. Make sure that Data API access is enabled by clicking on the button *Enable Data API access*.

Please note that mLab’s Data API uses [MongoDB Extended JSON in strict mode](https://docs.mongodb.com/v3.2/reference/mongodb-extended-json/) to encode queries and documents.


### Create a new Database

Now we need to create a new Database through the [mLab web UI wizard](https://mlab.com/create/wizard). 

1. Choose the SANDBOX plan type, which is free for storing up to 500MB of data and click on the button *Continue*.
2. Choose the Region where the new database will be located (e.g., Europe) and click on the button *Continue*.
3. Give a name to the database name (e.g., *adm*) and click on the button *Continue*.
4. The final page displays the details of the new database. Click *Submit Order* to create the new database.


### Simple operations via MongoDB Data API

We start by using the API endpoint */Databases* to retrieve the databases linked to the authenticated account using [requests](http://docs.python-requests.org/en/master/#) python library for using the HTTP protocol in a simple and straight-forward way.

**Make sure you replace your API key in the code below**

In [7]:
import requests

params = {'apiKey': '<Paste your api key HERE>'}
url = 'https://api.mlab.com/api/1/databases'
response = requests.get(url, params)

The [get](http://docs.python-requests.org/en/master/user/quickstart/#make-a-request) method is making an HTTP GET request passing the single parameter required by the particular API endpoint, that is the *apiKey*. 

If the HTTP request is completed successfully (the apiKey is correct, Data API access is enabled) the server will respond with 200 (OK).

In [8]:
response.status_code

200

We can access the contents of the result through the *text* element of the [response object](http://docs.python-requests.org/en/master/user/quickstart/#response-content)

In [19]:
response.text

'[ "adm" , "adm2017" , "ds" , "seed" ]'

In this example we used the name *adm* for the new database. 

We can retrieve the available collections of the new database through the following API:

In [20]:
dbname = 'adm2017'
url = 'https://api.mlab.com/api/1/databases/' + dbname + '/collections'
response = requests.get(url, params)

New database contain collections used by mongoDB.

In [21]:
response.text

'[ "objectlabs-system" , "objectlabs-system.admin.collections" , "system.indexes" ]'

Creating a new collection is very simple. We just add a new document to the particular collection and it is ready. We do this making a *POST* request to the corresponding API endpoint. As soon as you POST your first document you should see the collection appear.

Recall that mLab’s Data API uses [MongoDB Extended JSON in strict mode](https://docs.mongodb.com/v3.2/reference/mongodb-extended-json/) to encode queries and documents. So we will use the *dumps* method of JSON libray to convert the data into JSON format.

In [22]:
collection = 'laziodata'
url = 'https://api.mlab.com/api/1/databases/' + dbname + '/collections/' + collection
headers = {'content-type': 'application/json'}
data = json.dumps(dataset[0])
response = requests.post(url, data=data, params=params, headers=headers)

Given that the API call is complete successfully, the response will contain the record added along with a unique identity provided by mongoDB (under the field name *_id*). If you POST a document that contains an _id field, the effect will be to overwrite any existing document with that _id. 

In [23]:
response.text

'{ "csubingressi" : "1" , "cbar_ristoranti_chiusure" : "" , "cbar_ristoranti_aperture" : "" , "ctotale" : "32" , "cbar_ristoranti_subingressi" : "" , "cprovincia" : "VITERBO" , "cristoranti_subingressi" : "" , "ccomune" : "Acquapendente" , "canno" : "2007" , "cristoranti_totale" : "" , "cristoranti_chiusure" : "" , "cbar_chiusure" : "" , "cbar_totale" : "" , "cchiusure" : "0" , "cristoranti_aperture" : "" , "cbar_aperture" : "" , "cbar_ristoranti_totale" : "" , "caperture" : "0" , "cbar_subingressi" : "" , "_id" : { "$oid" : "59fa1b26c2ef162f47efffe3"}}'

### Insert multiple documents

For inserting multiple documents we can make one API call containing multiple records.

In [34]:
data = json.dumps(dataset[1:10])
response = requests.post(url, data=data, params=params, headers=headers)

Given that the API call is complete successfully, the response will contain the number of records added. 

In [25]:
response.text

'{ "n" : 9}'

#### Exercise
* Upload all the remaining records of the dataset to the collection.

### Query for documents

To get the documents in the specified collection we use the *List Documents* API endpoint. If no parameters are passed, it lists all of them. Otherwise, it lists the documents in the collection matching the specified parameters:
* q=<query> - restrict results by the specified JSON query
* c=true - return the result count for this query
* f=<set of fields> - specify the set of fields to include or exclude in each document (1 - include; 0 - exclude)
* fo=true - return a single document from the result set (same as findOne() using the mongo shell
* s=<sort order> - specify the order in which to sort each specified field (1- ascending; -1 - descending)
* sk=<num results to skip> - specify the number of results to skip in the result set; useful for paging
* l=<limit> - specify the limit for the number of results (default is 1000)

As an example let's retrieve the data that are related to the VITERBO province. We will use the *q* parameter to encode our query into a [query filter document](https://docs.mongodb.com/v3.2/core/document/#document-query-filter). Lookup the [documentation](https://docs.mongodb.com/v3.2/reference/operator/query/) for a detailed syntax of the query selectors.

In our example we wish to list all documents where the field "cprovincia" is "VITERBO". This is also encouded in JSON as follows:

 {"cprovincia": "VITERBO"}

In [35]:
query = {"cprovincia": "VITERBO"}

In [36]:
params['q'] = json.dumps(query)
response = requests.get(url, params=params)

The records that match the query are returned in JSON format. 

In [37]:
retrievedData = json.loads(response.text)

In [38]:
len(retrievedData)

420

We can narrow down the documents further by extending the query to a specific *ccomune*

In [39]:
query = {"cprovincia": "VITERBO", "ccomune": "Vitorchiano"}
params['q'] = json.dumps(query)
response = requests.get(url, params=params)

In [40]:
retrievedData = json.loads(response.text)

In [41]:
len(retrievedData)

7

### Update Documents

To update one or more documents in the specified collection, use a PUT request with a replacement document or update modifiers in the body (for more details see [MongoDB reference](https://docs.mongodb.com/v3.2/crud/#write-op-update)).

Once again we use the Query mechanism to narrow down the number of documents that will be updated.

In this example we wish to modify the 1st record from the data we just retrieved from the previous command by completely replacing the record.

In [42]:
retrievedData[0]['_id']

{'$oid': '59fa1b4dbd966f490aa55239'}

For the example we will change the cbar_totale value to '0'. 

In [43]:
retrievedData[0]['cbar_totale'] = 0

In [44]:
query = {"_id": retrievedData[0]['_id']}
params['q'] = json.dumps(query)
data = json.dumps(retrievedData[0])
response = requests.post(url, data=data, params=params, headers=headers)

The API call returns the new contents of the document

In [45]:
response.text

'{ "ctotale" : "16" , "_id" : { "$oid" : "59fa1b4dbd966f490aa55239"} , "cbar_ristoranti_aperture" : "" , "csubingressi" : "1" , "cprovincia" : "VITERBO" , "cristoranti_subingressi" : "" , "ccomune" : "Vitorchiano" , "canno" : "2007" , "cchiusure" : "1" , "cbar_ristoranti_totale" : "" , "cbar_ristoranti_subingressi" : "" , "cbar_chiusure" : "" , "cristoranti_totale" : "" , "cbar_subingressi" : "" , "cbar_aperture" : "" , "cristoranti_aperture" : "" , "cbar_totale" : 0 , "cbar_ristoranti_chiusure" : "" , "caperture" : "0" , "cristoranti_chiusure" : ""}'

### Retrieve Specific Document

We can retrieve a document directly based on it's unique id.

In [46]:
url = 'https://api.mlab.com/api/1/databases/' + dbname + '/collections/' + collection + "/" + retrievedData[0]['_id']["$oid"]

In [47]:
params.pop("query", None)

In [48]:
response = requests.get(url, params=params)

In [49]:
response.text

'{ "_id" : { "$oid" : "59fa1b4dbd966f490aa55239"} , "ctotale" : "16" , "cbar_ristoranti_aperture" : "" , "csubingressi" : "1" , "cprovincia" : "VITERBO" , "cristoranti_subingressi" : "" , "ccomune" : "Vitorchiano" , "canno" : "2007" , "cchiusure" : "1" , "cbar_ristoranti_totale" : "" , "cbar_ristoranti_subingressi" : "" , "cbar_chiusure" : "" , "cristoranti_totale" : "" , "cbar_subingressi" : "" , "cbar_aperture" : "" , "cristoranti_aperture" : "" , "cbar_totale" : 0 , "cbar_ristoranti_chiusure" : "" , "caperture" : "0" , "cristoranti_chiusure" : ""}'

### Delete Specific Document

We can delete a document directly based on it's unique id by making a DELETE request.

In [50]:
response = requests.delete(url, params=params)

In [51]:
response.text

'{ "_id" : { "$oid" : "59fa1b4dbd966f490aa55239"} , "ctotale" : "16" , "cbar_ristoranti_aperture" : "" , "csubingressi" : "1" , "cprovincia" : "VITERBO" , "cristoranti_subingressi" : "" , "ccomune" : "Vitorchiano" , "canno" : "2007" , "cchiusure" : "1" , "cbar_ristoranti_totale" : "" , "cbar_ristoranti_subingressi" : "" , "cbar_chiusure" : "" , "cristoranti_totale" : "" , "cbar_subingressi" : "" , "cbar_aperture" : "" , "cristoranti_aperture" : "" , "cbar_totale" : 0 , "cbar_ristoranti_chiusure" : "" , "caperture" : "0" , "cristoranti_chiusure" : ""}'

In [52]:
response = requests.get(url, params=params)

In [53]:
response.text

'{ "message" : "Document not found"}\n'